##### Run on master node, NOT IN EMR NOTEBOOK TERMINAL!!!
```
$ hdfs dfs -copyFromLocal /usr/lib/spark/external/lib/spark-avro.jar /apps/hudi/lib/
$ hdfs dfs -copyFromLocal /usr/lib/hudi/hudi-spark-bundle.jar /apps/hudi/lib/
```
##### Now start running paragraphs

In [1]:
%%configure -f
{
    "conf": {
                "spark.serializer":"org.apache.spark.serializer.KryoSerializer",
                "spark.sql.hive.convertMetastoreParquet":"false"
            },

    "jars": [
            "/apps/hudi/lib/hudi-spark-bundle.jar",
            "/apps/hudi/lib/spark-avro.jar"
        ]
}

In [2]:
import org.apache.spark.sql.SaveMode
import org.apache.spark.sql.functions._
import org.apache.hudi.DataSourceWriteOptions
import org.apache.hudi.config.HoodieWriteConfig
import org.apache.hudi.hive.MultiPartKeysValueExtractor

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
8,application_1574257904056_0006,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.sql.SaveMode
import org.apache.spark.sql.functions._
import org.apache.hudi.DataSourceWriteOptions
import org.apache.hudi.config.HoodieWriteConfig
import org.apache.hudi.hive.MultiPartKeysValueExtractor


In [5]:
val inputDataPath = "s3://athena-examples-us-west-2/elb/parquet/year=2015/month=1/day=1/"
val hudiTableName = "elb_logs_hudi_cow"
val hudiTablePath = "s3://jwyant-hudi/" + hudiTableName

// Set up our Hudi Data Source Options
val hudiOptions = Map[String,String](
    DataSourceWriteOptions.RECORDKEY_FIELD_OPT_KEY -> "request_ip",
    DataSourceWriteOptions.PARTITIONPATH_FIELD_OPT_KEY -> "request_verb", 
    HoodieWriteConfig.TABLE_NAME -> hudiTableName, 
    DataSourceWriteOptions.OPERATION_OPT_KEY ->
        DataSourceWriteOptions.INSERT_OPERATION_OPT_VAL, 
    DataSourceWriteOptions.PRECOMBINE_FIELD_OPT_KEY -> "request_timestamp", 
    DataSourceWriteOptions.HIVE_SYNC_ENABLED_OPT_KEY -> "true", 
    DataSourceWriteOptions.HIVE_TABLE_OPT_KEY -> hudiTableName, 
    DataSourceWriteOptions.HIVE_PARTITION_FIELDS_OPT_KEY -> "request_verb", 
    DataSourceWriteOptions.HIVE_ASSUME_DATE_PARTITION_OPT_KEY -> "false", 
    DataSourceWriteOptions.HIVE_PARTITION_EXTRACTOR_CLASS_OPT_KEY ->
        classOf[MultiPartKeysValueExtractor].getName)

// Read data from S3 and create a DataFrame with Partition and Record Key
val inputDF = spark.read.format("parquet").load(inputDataPath)

// Write data into the Hudi dataset
inputDF.write.
    format("org.apache.hudi").
    options(hudiOptions).
    mode(SaveMode.Overwrite).
    save(hudiTablePath)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

inputDataPath: String = s3://athena-examples-us-west-2/elb/parquet/year=2015/month=1/day=1/
hudiTableName: String = elb_logs_hudi_cow
hudiTablePath: String = s3://jwyant-hudi/elb_logs_hudi_cow
hudiOptions: scala.collection.immutable.Map[String,String] = Map(hoodie.datasource.write.precombine.field -> request_timestamp, hoodie.datasource.hive_sync.partition_fields -> request_verb, hoodie.datasource.hive_sync.partition_extractor_class -> org.apache.hudi.hive.MultiPartKeysValueExtractor, hoodie.datasource.hive_sync.table -> elb_logs_hudi_cow, hoodie.datasource.write.operation -> insert, hoodie.datasource.hive_sync.enable -> true, hoodie.datasource.write.recordkey.field -> request_ip, hoodie.table.name -> elb_logs_hudi_cow, hoodie.datasource.hive_sync.assume_date_partitioning -> false, hoodie.datasource.write.partitionpath.field -> request_verb)
inputDF: org.apache.spark.sql.DataFrame = [request_timestamp: string, elb_name: string ... 17 more fields]


In [7]:
// Count the records (should be 10491958)
inputDF.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res23: Long = 10491958


##### Run in Hive
```
hive> use default;
hive> select count(*) from elb_logs_hudi_cow;
```

In [8]:
val requestIpToUpdate = "243.80.62.181"
val sqlStatement = s"SELECT elb_name FROM elb_logs_hudi_cow WHERE request_ip = '$requestIpToUpdate'"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

requestIpToUpdate: String = 243.80.62.181
sqlStatement: String = SELECT elb_name FROM elb_logs_hudi_cow WHERE request_ip = '243.80.62.181'


In [9]:
spark.sql(sqlStatement).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+
|    elb_name|
+------------+
|elb_demo_003|
+------------+



In [15]:
// Create a DataFrame with a single record and update column value
val updateDF = inputDF.filter(col("request_ip") === requestIpToUpdate).withColumn("elb_name", lit("elb_demo_001"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

updateDF: org.apache.spark.sql.DataFrame = [request_timestamp: string, elb_name: string ... 17 more fields]


In [16]:
// Write the DataFrame as an update to existing Hudi dataset
updateDF.write.
    format("org.apache.hudi").
    options(hudiOptions).
    option(DataSourceWriteOptions.OPERATION_OPT_KEY, DataSourceWriteOptions.UPSERT_OPERATION_OPT_VAL).
    mode(SaveMode.Append).
    save(hudiTablePath)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
spark.sql(sqlStatement).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+
|    elb_name|
+------------+
|elb_demo_001|
+------------+



In [22]:
// Write the DataFrame with an EmptyHoodieRecordPayload for deleting a record
updateDF.write.format("org.apache.hudi").
    options(hudiOptions).
    option(DataSourceWriteOptions.OPERATION_OPT_KEY, DataSourceWriteOptions.UPSERT_OPERATION_OPT_VAL).
    option(DataSourceWriteOptions.PAYLOAD_CLASS_OPT_KEY, "org.apache.hudi.EmptyHoodieRecordPayload").
    mode(SaveMode.Append).
    save(hudiTablePath)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
spark.sql(sqlStatement).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+
|elb_name|
+--------+
+--------+



In [ ]:
// Run in bash
$ /usr/lib/hudi/cli/bin/hudi-cli.sh
// Now in hudi-cli
hudi->connect --path s3://jwyant-hudi/elb_logs_hudi_cow/
hudi:elb_logs_hudi_cow->commits show
hudi:elb_logs_hudi_cow->commit rollback --commit {last commit here}

In [20]:
spark.sql(sqlStatement).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+
|    elb_name|
+------------+
|elb_demo_001|
+------------+

